In [465]:
%run dbs_CodeSnippes.py
%matplotlib inline

pd.set_option('display.float_format', '{:.2f}'.format)

##### USER INPUT REQUIRED

In [466]:
#### set parameter IntroPage - user input
tsteady = (75, 's')                         # tuple of plateau time and unit
ph_t90 = (20, 's')                          # tuple of response time of pH sensor and unit
para2_t90 = (70, 's')                       # tuple of response time of second sensor sensing one analyte of the corresponding acid/base pair and unit

# UNITS ALLOWED: ppm, mg/L
paraSum_conc =  (100., 'ppm')              # tuple of concentration of the total parameter and unit
ph_edit = [4., 9.5, 11.05, 7.6]            # list of pH values
para2_name = ('NH4+','NH3', 'TAN')         # specify the corresponding acid/base pair and the measured analyte. 
                                           # either choose from a pre-defined system and select either NH3 or NH4+ for TAN or H2S or HS- for TDS system. 
                                           # if another system should be specified, describe the system as a tuple the following way:
                                           #### ('measured analyte', 'corresponding pair', 'total parameter') ###
para2_pka = 9.43                           # pKa value or equilibirum constant for given corresponding acid/base pair

response_curve = 'individual'              # specify which sensor response curve shall be used. use either 'default' or 'individual'.
                                           # when default is used, the standard Gompertz reationship is used; otherwise the user is welcome to specify 
                                           # their own response curves. In the latter case, additional properties and parameters are  required to fit 
                                           # in the provided pipeline of the code. Please refer to the manual for inidividual sensor response curves.

In [467]:
#### USER INPUT · Define sensor response curve
if response_curve == 'default':
    # pH sensor response curve and related parameter settings    
    func_resp_pH = _gompertz_curve_v1
    presponse_ph = dict({'t90': sensor_ph['t90'], 'tau': sensor_ph['resolution'], 'slope': 'increase'})
    
    # sensor response curve for the measurement sensor and related parameter settings    
    func_resp_meas = _gompertz_curve_v1
    presponse_meas = dict({'t90': sensor_para2['t90'], 'tau': sensor_para2['resolution'], 'slope': 'increase'})

elif response_curve == 'individual':
    print('USER INPUT REQUIRED. Please, specify your sensor response and respective parameters her before continuing.')
    # pH sensor response curve and related parameter settings    
    func_resp_pH = None
    presponse_ph = None 
    
    # sensor response curve for the measurement sensor and related parameter settings    
    func_resp_meas = None
    presponse_meas = None
    
else:
    raise ValueError('Please do not confuse the algorithm. \nEither use *default* when the Gompertz curve shall be used as sensor response.'+
                     'Otherwise set response_curve as *individual* and define your response curve and respective parameters above.')

USER INPUT REQUIRED. Please, specify your sensor response and respective parameters her before continuing.


##### Run Simulation

In [468]:
###### Prepare parameters
# get the names and parameter labels for the specified system
[para2_grp, para3_grp, total_para_grp, para2, para3, total_para, analyte] = _getAnalyteSystem(para2_name)

# check if all required variables are defined and have the same units 
[sensor_ph, sensor_para2, 
 para_meas] = check_parameter(analyte=analyte, paraSum_conc_edit=str(paraSum_conc[0]), ph_edit=str(ph_edit)[1:-1], 
                              para2_pka_edit=para2_pka, tsteady_edit=tsteady, ph_t90_edit=ph_t90, para2_t90_edit=para2_t90)

In [469]:
##### RUN the simulation
if float(sensor_ph['t90']) == 0:
    raise ValueError("WARNING: Please enter a valid sensor response time for the pH sensor, in particular a number > 0")
else:
    if float(sensor_para2['t90']) == 0:
        raise ValueError("WARNING: Please enter a valid sensor response time for the pH sensor, in particular a number > 0")
    else:
        if func_resp_pH and func_resp_meas:
            # in case parameter check was successful, start run_simulation
            [df_res, ls_lines, ls_xcoords, 
            vlines_list] = run_simulation_divResponses(sensor_ph=sensor_ph, sensor_para2=sensor_para2, para_meas=para_meas, paraSum_unit=paraSum_conc[1], 
                                                        func_resp_pH=func_resp_pH, presponse_ph=presponse_ph, func_resp_meas=func_resp_pH, 
                                                        presponse_meas=presponse_meas) 
        else:
           raise ValueError("WARNING: Please make sure that all sensor response curves are specified.") 
df_res.head()

ValueError: WARNING: Please make sure that all sensor response curves are specified.

### Plot Results

In [472]:
### plot pH and acid base pair (target vs sensor response)
if df_res:
    fig_ph, ax_ph = plot_phSensor(df_res, fgs=(5.5, 3.25))
    fig_pair, ax_pair, ax_pair1 = plot_AcidBasePair(para2_grp, para3_grp, df_res, fgs=(5.5, 3.25))

In [473]:
## total parameter (target vs sensor response)
if df_res:
    # Figure 1 · Overview displaying the entire error/offset
    fig_sum, ax_sum = plot_totalParameter(total_para_grp, total_para, df_res, ylim=None, fgs=(5.5, 3.5))

    # Figure 2 · Zoom-in +/-10% around target concentration
    fig_sum, ax_sum = plot_totalParameter(total_para_grp, total_para, df_res, ylim='10%', fgs=(5.5, 3.5))

#### Determine error via integration

In [475]:
### USER-INPUT · define the integration range that shall be analysed
ls_xcoords = [(75, 245), (245, 300)]

# -----------------------------------------------------------------------------------------------------------------------
res_integral = None
if df_res:
    # calculate the error within given integral ranges 
    res_integral = calculate_integralError(ls_xcoords=ls_xcoords, df_res=df_res, sensor_ph=sensor_ph, 
                                           sensor_para2=sensor_para2, para2_name=para2_name, paraSum_conc=paraSum_conc)
res_integral